In [1]:
## WARMUP
from collections import Counter
def most_common_letters(sentence):
    """
    INPUT: string
    OUTPUT: list of strings

    Given a sentence, give the most common letter for each word.
    You should lowercase the letters. If there's a tie, include any of them.

    example:
    INPUT: "Welcome to Zipfian Academy!"
    OUTPUT: 'e t i a'

    Hint: use Counter and the string join method
    (It is possible to do this in one line, but you might lose some
    readability)
    """
    answer_string = ""
    sentence=sentence.lower()
    word_list = sentence.split(" ")
    for word in word_list:
        cnt = Counter()
        for char in word:
            cnt[char] += 1
        answer_string += cnt.most_common()[0][0]+" "
    return answer_string.strip()


In [2]:
most_common_letters("If you had a twin, I would still choose you")

'f u h a , i d l o u'

# Week 4 -- SQLAlchemy
In this week's lab you will be building the database for an E-Commerce website called [FizzBuzz Cola](http://fizzbuzzcola.herokuapp.com/). 

### Database Layout
<Image goes here>

The first thing we should do when creating a database is map out all of the tables we want to create and how they will interact with each other.  Check out this image to see how our final database layout will look. You may also want to refer to the [SQLAlchemy Documentation](http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html) to understand the basics of setting up database relationships with SQLAlchemy.

![Database Layout](fizzbuzz.jpg)

## Part 1 -- Setting up SQLAlchemy
We must first import some necessary librares to use SQLAlchemy in our project. After all of the imports we use the declarative_base method in SQLAlchemy to make a Base class. 

In [ ]:
from sqlalchemy import Column,Integer,String, DateTime, ForeignKey, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy import create_engine, func

Base = declarative_base()

## Part 2 -- Setting up the Products Table
All of the tables we want to create will be represent as Python _objects_ that _inhert_ from the __Base__ _class_ (familiarize yourself with these terms when referring to Object-Oriented Programming) 

Each _object_ that inherits from __Base__ must have a "__tablename__" variable specified. By convention, we make this a lower-case version of the original class name. Then we add each column along with the type of information we want to store in that column.  To see all of the different types of data we can store, refer to the SQLAlchemy documentation.

In [ ]:
class Product(Base):
    __tablename__ = 'product'
    __table_args__ = {'extend_existing': True}  #This will enable us to add more columns later
    id = Column(Integer, primary_key=True)
    name = Column(String)
    description = Column(String)
    photo = Column(String)
    price = Column(String)
    

Once we are done specifying our tables, we now must create an engine which specifies our connection to the database along with a DBSession which sends our python operations to the engine to be processed into SQL.  

This command will create an SQLite mini-database in the same directory as this notebook. Check and see that it is created after running the code below.

In [ ]:
engine = create_engine('sqlite:///fizzBuzz.db')
Base.metadata.create_all(engine)

DBSession = sessionmaker(bind=engine, autoflush=False)
session = DBSession()

# CRUD - Create, Read, Update, Delete

The four most basic operations we do on a database are summarized with the CRUD acronym. We will now see how to perform CRUD operations on our database using SQLAlchemy

# CREATE

Now that our database is created, let's add a new item to it.  Look at the code below to see how we add a new product to our database.  

In [ ]:
newProduct = Product(name="Cream Soda", description="Cream soda is a sweet carbonated soft drink flavored with vanilla.", photo='http://www.caffeineinformer.com/wp-content/caffeine/a-w-cream-soda.jpg', price='0.99')
session.add(newProduct)
session.commit()

Add another type of soda to the database. Write your code below. You can search on the internet for a .jpg or .png image of your soda. 

# READ
In order to read information out of our database we create queries.  A quiery will look inside of a specific table and grab one or many rows of data based on the criteria we request.  In the code below we are requesting the first row from the Product table

In [ ]:
query = session.query(Product).first()

We can see the contents of the query by printing the values :

In [ ]:
print(query.name)
print(query.id)
print(query.price)
print(query.photo)
print(query.description)


#We can display the actual photo like this
from IPython.display import Image
from IPython.core.display import HTML 

Image(url= query.photo)

Try using the filter_by function to find the soda you created, the id should be 2. Then print out the price, photo, and description of your soda from the database

In [ ]:
#Use the filter_by method to find your soda based on its id

In [ ]:
#Print the price, photo and description here

# UPDATE
Updating allows us to change specific values in a row without deleting or creating a new row in our table.  In order to update a row we must first execute a query to find the row we are looking for:

In [ ]:
query = session.query(Product).filter_by(id=1).one()

Then we can update any of the parameters by reassigning the variables like so:

In [ ]:
query.name = "A&W Cream Soda"
query.photo = "http://www.governorsrecipes.com/images/flavor_cream_soda.jpg"

We then add and commit the session to the database

In [ ]:
session.add(query)

In [ ]:
session.commit()

Re-running this code should now read the updated entries from our database:

In [ ]:
print(query.name)
print(query.id)
print(query.price)
print(query.photo)
print(query.description)


#Display the actual photo here
Image(url= query.photo)

Update the price of the soda entry you created and show it below:

## Delete
Similarly to updating, we delete a row from our database by first executing a query:

In [ ]:
query = session.query(Product).filter_by(id=1).one()

...then by calling session.delete() and commiting we have executed that deletion

In [ ]:
session.delete(query)
session.commit()

So what do you expect to happen when we search for an entry in a database we deleted?

In [ ]:
query = session.query(Product).filter_by(id=1).one()

In [ ]:
#Delete the soda object your created from the database

## Adding A Customer Table

Now that we have figured out how to create products in our database, let's add some customers as well. Look at and execute the code below, you'll notice a few new features that we've added to our class.

In [ ]:
from passlib.apps import custom_app_context as pwd_context

class Customer(Base):
    __tablename__ = 'customer'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    address = Column(String(255))
    email = Column(String(255), unique=True)
    password_hash = Column(String(255))
    shoppingCart = relationship("ShoppingCart", uselist=False, back_populates="customer")
    orders = relationship("Order", back_populates="customer")

    def hash_password(self, password):
        self.password_hash = pwd_context.encrypt(password)

    def verify_password(self, password):
        return pwd_context.verify(password, self.password_hash)


The first thing you should notice is a password hash. Hashing is a special one-way mathematical function that we can use to safely store sensitve information in a database.  Dictionaries also use a special form of hashing in Python.  

We have also added Relationships to a Shopping Cart (1-to-1) as well as to orders (many-to-many)

Many-to-Many relationships require an intermediate table to be created but we will get to that very soon. 

# Adding A Shopping Cart
So as we previously stated, every customer should have one Shopping Cart, But each Shopping Cart may have mulitple products and products can be in multiple shopping carts.  Read up on creating one-to-one and many-to-many relationships in the SQLAlchemy Documentation and then read through the code below.

In [ ]:
class ShoppingCart(Base):
    __tablename__ = 'shoppingCart'
    id = Column(Integer, primary_key=True)
    customer_id = Column(Integer, ForeignKey('customer.id'))
    customer = relationship("Customer", back_populates="shoppingCart")
    products = relationship("ShoppingCartAssociation", back_populates="shoppingCart")

# Updating Relationships in Products
Our products now also must be adapted to belong to several shopping carts as well as final orders. So let's update the Product class with these two relationships. 

In [ ]:
class Product(Base):
    __table_args__ = {'extend_existing': True}
    __tablename__ = 'product'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    description = Column(String)
    photo = Column(String)
    price = Column(String)
    orders = relationship("OrdersAssociation", back_populates="product")
    shoppingCarts = relationship("ShoppingCartAssociation", back_populates="product")


## Order Table
When a customer checks out, we will clear their shopping cart, place an order containing all of the products, and provide a confirmation number for their order. 

In [ ]:
class Order(Base):
    __tablename__ = 'order'
    id = Column(Integer, primary_key=True)
    total = Column(Float)
    timestamp = Column(DateTime, default=func.now())
    confirmation = Column(String, unique=True)
    products = relationship("OrdersAssociation", back_populates="order")
    customer_id = Column(Integer, ForeignKey('customer.id'))
    customer = relationship("Customer", back_populates="orders")

# Intermediate Tables:

When we have many-to-many relationships, intermediate or joint tables must be created to keep track of the foreign key relationships as well as to specify the rules as to what happens if an item is deleted from one table and not another.  Read through the code below and see if you can understand what's going on with the two intermediate tables below.  If you have any questions find a TA or Instructor.  

In [ ]:
class OrdersAssociation(Base):
    __tablename__ = 'OrdersAssociation'
    order_id = Column(Integer, ForeignKey('order.id'), primary_key=True)
    product_id = Column(Integer, ForeignKey('product.id'), primary_key=True)
    product_qty = Column(Integer)
    product = relationship("Product", back_populates="orders")
    order = relationship("Order", back_populates="products")

class ShoppingCartAssociation(Base):
    __tablename__ = 'shoppingCartAssociation'
    shopping_cart_id = Column(Integer, ForeignKey('shoppingCart.id'), primary_key=True)
    product_id = Column(Integer, ForeignKey('product.id'), primary_key=True)
    quantity = Column(Integer)
    product = relationship("Product", back_populates="shoppingCarts")
    shoppingCart = relationship("ShoppingCart", back_populates="products")


Now that we've added all these database modifications, let's update our database to reflect these changes in our python code (If an error occurs at this step try either deleting the old fizzBuzz.db file or re-naming the one we create below)

In [ ]:
engine = create_engine('sqlite:///fizzBuzz.db')
Base.metadata.create_all(engine)

DBSession = sessionmaker(bind=engine, autoflush=False)
session = DBSession()

# Populate Products

Now that we're done with all our database configurations, let's populate it with a few items. See how we can use a list of dictionaries to easily add multiple items. If you've ever heard of JSON this probably looks familar. If you haven't, don't worry, you will all be familiar with JSON very soon.

Feel free to add any other items if you'd like. 

In [ ]:
products = [
    
{'name':'Lemon Lime', 'description':'Obey your Thirst', 'photo':'https://i5.walmartimages.com/asr/f6bbc322-83c3-49d9-a9b6-35f05aea0226_1.e37e538746a60bad395e7a0b19ab4f6c.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF', 'price':'$2.99'},
{'name':'Tutti Fruiti', 'description':'Tropical Fruit Punch', 'photo':'https://i5.walmartimages.com/asr/859eac0f-f23f-4bf3-b190-91a97d495bbe_1.1376acaadf8d89cb4a12f42fd0318b53.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF', 'price':'$1.89'},
{'name':'Root Beer', 'description':'Made from Sassafras', 'photo':'http://cdn6.bigcommerce.com/s-vs756cw/products/1114/images/1739/Tower_Root_Beer__34890.1448901409.1280.1280.png?c=2', 'price':'$1.50'},
{'name':'Strawberry', 'description':'Not Fanta, but the next best thing', 'photo':'http://texaslegacybrands.com/media/catalog/product/cache/1/image/800x/9df78eab33525d08d6e5fb8d27136e95/n/e/nesbitt-040190.jpg', 'price':'$.99'},
{'name':'Traditional Cola', 'description':'A Traditional Favorite', 'photo':'https://i5.walmartimages.com/asr/d6ae552d-5bf8-4fcb-9a2f-3be899a90024_1.1da92d09e8dd1ecf04a0d178a909c5cc.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF', 'price':'$.88'},
{'name':'Grape', 'description':'Fresh off the vine goodness', 'photo':'http://www.zandh.co.uk/media/catalog/product/cache/1/image/600x600/9df78eab33525d08d6e5fb8d27136e95/o/l/old_jamaica_grape_soda.png', 'price':'$1.29'},
{'name':'Orange', 'description':'Zesty citrus flavor', 'photo':'http://edengourmet.com/wp-content/uploads/2014/10/Boylans-Orange-Soda-12floz.jpg', 'price':'$2.15'},
{'name':'Peach', 'description':'Fuzzy Navel of Refreshment', 'photo':'https://s3.amazonaws.com/static.caloriecount.about.com/images/medium/big-peach-soda-84751.jpg', 'price':'$1.99'},
{'name':'Diet Cola', 'description':'Same great taste without the calories', 'photo':'http://acttwomagazine.com/wp-content/uploads/2015/07/Diet_Cola1.jpg', 'price':'$1.00'},
{'name':'Energy Cola', 'description':'Gives you wings', 'photo':'https://jarrodhart.files.wordpress.com/2011/09/generic_energy_drink.jpg', 'price':'$2.99'},
        
]


for product in products:
    newProduct = Product(name=product['name'], description=product['description'], photo=product['photo'], price=product['price'])
    session.add(newProduct)
    session.commit()

#### Now, let's print a catalog of our inventory:

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display, Image

queries = session.query(Product).all()
for query in queries:
    print(query.name)
    print(query.id)
    print(query.price)
    display(Image(url= query.photo,width=100, height=100))
    print(query.description)
    print("\n")

    

# Create two New Customers
Modify the code below to add you and your partner as customers in the database.  What happens when I try to retrieve a password from the database.  Then what happens when I verify a password.

In [ ]:
customer1 = Customer(name = "Partner 1 Name", email="partner1@meet.mit.edu", address = "3 Lincoln St.")
password="my_super_secret_password"
customer1.hash_password(password)
session.add(customer1)
shoppingCart = ShoppingCart(customer=customer1)
session.add(shoppingCart)
session.commit()

customer2 = Customer(name = "Partner 2 Name", email="partner2@meet.mit.edu", address = "3 Lincoln St.")
password="another_super_secret_password"
customer1.hash_password(password)
session.add(customer2)
shoppingCart = ShoppingCart(customer=customer2)
session.add(shoppingCart)
session.commit()

In [ ]:
customer1.password_hash


In [ ]:
customer1.verify_password("my_super_secret_password")

#  Add Items to Shopping Carts
Now let's see if we can add an item to the shopping cart. To do this let's make a reusable function that will add an item based on the customer's id (which is the same as the shopping cart's id), the product_id, and the quantity)

In [ ]:
def add_to_cart(customer_id,product_id, quantity):
    product = session.query(Product).filter_by(id=product_id).one()
    shoppingCart = session.query(ShoppingCart).filter_by(customer_id = customer_id).one()
    customer = session.query(Customer).filter_by(id = customer_id).one()
    #If this item is already in the shopping cart, just update the quantity
    if product.name in [item.product.name for item in shoppingCart.products]:
        assoc = session.query(ShoppingCartAssociation).filter_by(shoppingCart=shoppingCart).filter_by(product = product).one()
        assoc.quantity = int(assoc.quantity) + int(quantity)
    else:
        a = ShoppingCartAssociation(product = product, quantity=quantity)
        shoppingCart.products.append(a)
        session.add_all([a,product,shoppingCart])
        session.commit()
    print("Successfully added %s %s to %s's Shopping Cart"% (str(quantity), str(product.name),str(customer.name) ))

Try adding these and a few more items to the cart (at least 5 more items)

In [ ]:
add_to_cart(1,3,5)
add_to_cart(1,2,6)
add_to_cart(2,1,3)
add_to_cart(2,7,15)

# Remove an Item from Shopping Cart
We we remove an item from a customer's shopping cart, we are only removing the relationship between that item and the shopping cart in the ShoppingCartAssociation table.  See the code below and understand what it is doing

In [ ]:
def removeFromCart(customer_id, product_id):
    customer = session.query(Customer).filter_by(id = customer_id).one()
    product = session.query(Product).filter_by(id=product_id).one()
    association = session.query(ShoppingCartAssociation).filter_by(shopping_cart_id=customer_id).filter_by(product_id = product_id).one()
    session.delete(association)
    session.commit()
    print("%s Deleted Successfully from %s's Cart"%(product.name, customer.name))


In [ ]:
removeFromCart(1,3)
removeFromCart(2,1)

# Update an Item Quantity in Shopping Cart

Similarly to deleting, updating only requires modifing the quantity field in the ShoppingCartAssociation table:

In [ ]:
def updateQuantity(customer_id, product_id, quantity):
    customer = session.query(Customer).filter_by(id = customer_id).one()
    product = session.query(Product).filter_by(id=product_id).one()
    if quantity == 0:
        return removeFromCart(customer_id, product_id)
    if quantity < 0:
        print("Can't store negative quantities because that would be silly.")
        return
    association = session.query(ShoppingCartAssociation).filter_by(shopping_cart_id=customer_id).filter_by(product_id = product_id).one()
    association.quantity = quantity
    session.add(association)
    session.commit()
    print("%s's Quantity in %s's Cart Updated Successfully to %s" % (product.name, customer.name, str(quantity)))

Update some of the other items in your shopping carts

In [ ]:
updateQuantity(1,2,7)
updateQuantity(2,7,7)

# Place an Order
When we are finished shopping, we much checkout and clear our shopping cart.  The code below in ```checkout``` loops through each item in our shopping cart and calculates the total price. The ```generateConfirmationNumber``` creates a random unique string for our customers to track their orders.  

In [ ]:
import random, string
def generateConfirmationNumber():
    confirmation = ''.join(random.choice(string.ascii_uppercase + string.digits) for x in range(16))
    if confirmation in [order.confirmation for order in session.query(Order).all()]:
        generateConfirmationNumber()
    return confirmation

def checkout(customer_id):
        shoppingCart = session.query(ShoppingCart).filter_by(customer_id = customer_id).one()
        order = Order(customer_id = customer_id, confirmation = generateConfirmationNumber())
        total = 0.00
        for item in shoppingCart.products:
            assoc = OrdersAssociation(product = item.product, product_qty = item.quantity)
            order.products.append(assoc)
            total += float(item.product.price.strip('$')) * int(item.quantity)
            session.delete(item)
        order.total = total
        session.add_all([order, shoppingCart])
        session.commit()
        print("Your order has been placed with confirmation %s. Your total is $%s" % (order.confirmation, str(total)))
   

Checkout all of the customers in your database. What happens when you try to checkout twice? Can you think of a more elegant solution?

In [ ]:
checkout(1)

In [ ]:
checkout(2)

## Extra Challenge - Keeping Track of Inventory
In a real e-commerce scenario, we don't want to sell things that are out-of-stock to our customers.  Add the necessary database tables to keep track of inventory. Add the necessary python code to make sure that a customer cannot checkout or add items to their shopping cart if the inventory for that item is too low.